Split the dataset into training (80%) and test (20%) sets in 5 different folds. Train the simple linear regression model (using gradient descent) for each split on the train-test data in each fold. Evaluate the model on the test set in each fold using:

- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- R2 score